# Chapter 1: Transparent Conductors

### Instruction: The purpose of this notebook is to try and find values of electron density and mean free time for a material suitable to be used as a transparent conductors. The final estimate should be the order of the two parameters to be used

In [ ]:
import numpy as np

from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure
output_notebook()

from ipywidgets import interact

from collections import OrderedDict
old_settings = np.seterr(over = 'ignore') #Ignore warnings about overflow data points

#### First, we set up initial variables used in the calculation. These involve defining electron density, electron mass and charge, material resistivity/conductivity, vacuum and effective material permittivity, the bounds of frequency used in calculations and the number of points we want to calculate for our graphs. Additionally, mean free time tau can be calculated from 
$\sigma = ne^2\tau /m $

In [ ]:
# Set up constant in SI units
freq_min = 1e+14   #Lower limit of frequency
freq_max = 10e+14    #Upper limit of frequency
dens = 8.47e+28 #Define initial electron density
m = 9.109e-31 #Define electron mass
elec = 1.602e-19 #Define elementary electron charge
rho = 1.56e-8 #Define DC resistivity
sigma0 = 1/rho #Calculate initial DC conductivity from resisitivity
eps = 8.854e-12 #Vacuum permittivity
eps0 = 1.0 #Effective permittivity of the material
N = 5000   #Number of points for the independent variable
tau = m / (dens*rho*elec**2) #Calculate mean free time

#### Next, we define a variable containing all our frequency points and convert frequecy to angular frequency. With all variables defined, we can calculate the real and imaginary part of the AC conductivity.

In [ ]:
# Set up the variable space
freq = np.linspace(freq_min, freq_max, N)  #Energy measured in units of chemical potential
w = 2*np.pi*freq #Convert frequency to angular frequency
# Calculate AC conductivity
sigma1 = sigma0 / (1.0 + w**2*tau**2)  #Real part of the conductivity
sigma2 = w*tau*sigma0 / (1.0 + w**2*tau**2)  #imaginary part of the conductivity

#### With AC coductivity calculated, we can proceed to calculate the permittivity of the material.

In [ ]:
#calculate permittivity
eps1 = eps0 - sigma2 /(eps*w)
eps2 = sigma1/(eps*w)

#### In the two inputs below we set up the plot figures four our 4 graphs (imaginary and real part of conductivity, imaginary and real part of permittivity)

In [ ]:
# Set up conductivity plot
plot = figure(plot_height=200, plot_width=400, title="Real part of AC conductivity",
              tools="pan,reset,save,wheel_zoom",
              x_range=[freq_min, freq_max])
plot.xaxis[0].axis_label='Frequency (Hz)'
plot.yaxis[0].axis_label='conductivity (Ohm-1m-1)'
r = plot.line(freq, sigma1, line_width=3, line_alpha=0.6)

plot2 = figure(plot_height=200, plot_width=400, title="Imaginary part of AC conductivity",
              tools="pan,reset,save,wheel_zoom",
              x_range=[freq_min, freq_max], y_range=[0.0, 2e+7])
plot2.xaxis[0].axis_label='Frequency (Hz)'
plot2.yaxis[0].axis_label='conductivity (Ohm-1m-1)'
rr = plot2.line(freq, sigma2, line_width=3, line_alpha=0.6, color = 'red')

In [ ]:
# Set up permittivity plot
plot3 = figure(plot_height=200, plot_width=400, title="Real part of permittivity",
              tools="pan,reset,save,wheel_zoom",
              x_range=[freq_min, freq_max])
plot3.xaxis[0].axis_label='Frequency (Hz)'
plot3.yaxis[0].axis_label='Permittivity (F/m)'
rrr = plot3.line(freq, eps1, line_width=3, line_alpha=0.6)

plot4 = figure(plot_height=200, plot_width=400, title= "imaginary part of permittivity",
              tools="pan,reset,save,wheel_zoom",
              x_range=[freq_min, freq_max])
plot4.xaxis[0].axis_label='Frequency (Hz)'
plot4.yaxis[0].axis_label='Permittivity (F/m)'
rrrr = plot4.line(freq, eps2, line_width=3, line_alpha=0.6, color = 'red')

#### As final step, we define an update function which would allow us to use sliders to change input variables (such as density and DC resistivity) and see the change in the plot without having to rerun the code.

In [ ]:
# Set up callbacks to update the plots
def update_data(density_pow, mft_pow):
# Generate the new curve
    dens = 10**density_pow
    tau = 10**mft_pow
    rho = m / (dens*tau*elec**2)
    sigma0 = 1/rho
    freq = np.linspace(freq_min, freq_max, N)  #Energy measured in units of chemical potential
    w = 2*np.pi*freq #Convert frequency to angular frequency
    r.data_source.data['y'] = sigma0 / (1.0 + w**2*tau**2)
    rr.data_source.data['y'] = w*tau*sigma0 / (1.0 + w**2*tau**2)
    rrr.data_source.data['y'] = eps0 - rr.data_source.data['y']/(eps*w)
    rrrr.data_source.data['y'] =  r.data_source.data['y']/(eps*w)
    print('DC Conductivity =', f'{sigma0:.3}', 'S/m')

    push_notebook()
    
def update_data_prec(density_pow, mft_pow):
# Generate the new curve
    dp = 1 ###  Replace with your guess of order of magnitude for electron density
    tp = 1 ###  Replace with your guess of order of magnitude for mean free time
    dens = density_pow*10**dp
    tau = mft_pow*10**tp
    rho = m / (dens*tau*elec**2)
    sigma0 = 1/rho
    freq = np.linspace(freq_min, freq_max, N)  #Energy measured in units of chemical potential
    w = 2*np.pi*freq #Convert frequency to angular frequency
    r.data_source.data['y'] = sigma0 / (1.0 + w**2*tau**2)
    rr.data_source.data['y'] = w*tau*sigma0 / (1.0 + w**2*tau**2)
    rrr.data_source.data['y'] = eps0 - rr.data_source.data['y']/(eps*w)
    rrrr.data_source.data['y'] =  r.data_source.data['y']/(eps*w)
    print('DC Conductivity =', sigma0, 'S/m')

    push_notebook()

#### We can now plot the data

### Instruction:  In the code below, the last line introduces sliders for density (denstiy_pow) and mean free time (mft_pow). The way it is set up right now, the sliders provide ability to change order of magnitude of these parameters. You should set up the ranges for these sliders yourself. (just change the 4 variables below, where indicated)

### As a reference, conductivity of copper at room temperature is 5.96×10^7 S/m

### Not required, but if interested, changing update_data to update_data_prec in code below would allow the ranges to be linear, but your guess for the order of magnitude for two parameters would have to go into the formula above. This way you could try to get a better accuracy for the two parameters.

In [ ]:
den_min_range = 1 #Replace with minimum for the range of electron density order of magnitude (e.g. 2 corresponds to n = 10^2)
den_max_range = 1 #Replace with maximum for the range of electron density order of magnitude
mft_min_range = 1 #Replace with minimum for the range of mean free time (tau) order of magnitude
mft_max_range = 1 #Replace with maximum for the range of mean free time (tau) order of magnitude 

show(column([row(plot,plot2),row(plot3, plot4)]),notebook_handle=True)
interact(update_data, density_pow = (den_min_range, den_max_range, 1), mft_pow = (mft_min_range, mft_max_range, 1))